
<img src="https://upload.wikimedia.org/wikipedia/commons/7/72/YouTube_social_white_square_%282017%29.svg">


# `Youtube`-ETL



#

In [20]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.


##### Importing the necessary packages

In [1]:
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine

##### Get the playlist id 

In [37]:

api_key = "your_api_key"
channel_id="UC-lHJZR3Gqxm24_Vd_AJ5Yw"
youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=api_key)

def getPlaylistId() :
    
    request = youtube.channels().list(
            part='snippet,contentDetails,statistics',
            id=channel_id)

    response = request.execute()

    for channel in response["items"] :
        
        return channel["contentDetails"]["relatedPlaylists"]["uploads"] 
    

##### Get the data 

In [38]:

def uploadVideos():
    
    playlistId = getPlaylistId()
    
    request = youtube.playlistItems().list(
        part = "snippet,contentDetails",
        playlistId = playlistId,
        maxResults = 50
    )

    response = request.execute()

    publishedAt = []
    title = []
    video_id = []
    views = []
    likes = []

    for channel in response["items"] :
        publishedAt.append(channel["snippet"]["publishedAt"][0:10])
        title.append(channel["snippet"]["title"])
        video_id.append(channel['snippet']['resourceId']['videoId'])

    for idV in video_id :
        video_request = youtube.videos().list(
            part="statistics",
            id=idV
        )
        video_response = video_request.execute()
        views.append(video_response["items"][0]["statistics"]["viewCount"])
        likes.append(video_response["items"][0]["statistics"]["likeCount"])


    data = {
        "Title": title,
        "PublishedAt": publishedAt,
        "ViewsCounts": views,
        "LikeCounts" : likes
        
    }
    
    return data


##### Transform it into a dataframe 

In [39]:
df = pd.DataFrame(uploadVideos())

In [40]:
df.head()

,Title,PublishedAt,ViewsCounts,LikeCounts
0,I Learned 7 things in 7 days (to impress my son),2023-10-12,1542365,118374
1,This is the best,2023-10-04,3158367,250949
2,The worst Hoarder on the Planet.,2023-09-30,1533686,82256
3,i am now supreme leader,2023-09-22,1646848,120525
4,dad life,2023-09-15,3804573,251355


##### save it into a `csv` file 

In [41]:
df.to_csv("PewDiePie_data.csv", index = False)

##### Connect to `mysql` database and create the table  

In [47]:

db_user = 'user'
db_password = 'password'
db_host = 'localhost'
db_name = 'youtube_etl'
        
        
def createTable():
    
    try :
        
        connection = mysql.connector.connect(
                user=db_user,
                password=db_password,
                host=db_host,
                database=db_name
            )

        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version", db_Info)
            cursor = connection.cursor()
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database:", record)

        cursor = connection.cursor()

        sql_query = """
        DROP TABLE PewDiePie_data;
        CREATE TABLE PewDiePie_data (
            Title VARCHAR(200),
            PublishedAt VARCHAR(200),
            ViewsCounts VARCHAR(200),
            LikeCounts VARCHAR(200)
        );
        """

        cursor.execute(sql_query)
        print("Table successfully created")
            
            
    except mysql.connector.Error as err:
        print("MySQL Error:", err)


In [48]:
createTable()

Connected to MySQL Server version 8.0.33
You're connected to database: ('youtube_etl',)
Table successfully created


##### load the data into `mysql` database 

In [49]:

def load():
    try :
        db_connection_str = f'mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}'
        db_engine = create_engine(db_connection_str)
        df.to_sql('PewDiePie_data', con=db_engine, if_exists='append', index=False)
        db_engine.dispose()
        print("Data successfully inserted into the database")

    except mysql.connector.Error as err:
        print("MySQL Error:", err)


In [50]:
load()

Data successfully inserted into the database


# 








made by **Houda Moudni** 
Data engineering student at National School of Applied Science-ENSA
- **e-mail** : mdnhouda1702@gmail.com   
- **LinkedIn** : [Houda Moudni](https://www.linkedin.com/in/houda-moudni-525708257/)  

